In [155]:
# This script can be run to create a dataset of google news headlines. 

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs4

In [156]:
searchTerm = "something"
count=0
url = ("https://www.google.com/search?q=" + searchTerm + "&hl=en&gl=us&authuser=0&tbs=qdr:y&tbm=nws&start=$count$&cad=h")

headline = []
source_time = []

while (count<20):
    new_url = url.replace("$count$", str(count))
    r = requests.get(new_url)
    
    soup = bs4(r.text, 'html.parser')
    headline += soup.find_all("div", "st")
    source_time += soup.find_all("div", "slp")

    count = count+10 

In [157]:
def textify():

    source_time_text  = []
    for d in source_time:
        source_time_text.append(d.find('span').text)
        
    return print(source_time_text)

textify()

['ESPN - 14 hours ago', 'ABS-CBN News - 15 hours ago', 'MinnPost - 1 day ago', 'Rowan Today - 7 hours ago', 'New York Times - 3 days ago', 'NME.com - 1 day ago', 'KTRK-TV - 1 hour ago', 'Wichita Eagle - 1 day ago', 'KATC Lafayette News - 6 hours ago', 'Jalopnik - 1 day ago', 'Washington Post - 1 day ago', 'Deseret News - 4 hours ago', 'The Sydney Morning Herald - 14 hours ago', 'New York Magazine - 2 days ago', 'Vox - 1 day ago', 'Los Angeles Times - 6 hours ago', 'VG247 - 10 hours ago', 'The Independent (blog) - 16 hours ago', 'The Independent - 9 hours ago', 'Berkeley Independent - 10 hours ago']


In [162]:
def headlinify():
    
    headline_text = []
    for d in headline:
        headline_text.append(d.find('span').text)
        
    return print(headline_text)

headlinify()

AttributeError: 'NoneType' object has no attribute 'text'

In [163]:
type(headline)

list